<a href="https://colab.research.google.com/github/BMugo84/pytorch_in_25_hours/blob/main/08_pytorch_paper_replicating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Notes:  Reading Machine Learning Papers**

**Paper:** An image is worth 16x16 words (Attention is all you need)

**Transformers:** This is a deep-learning model that adopts the mechanism of self-attention, differentially weighing the significance of each part of input data (Wikipedia).

**Attention:** This is the most vivid part of an image that captures your eyes and delivers the image as a whole. E.g., a dog in the grass by a tree under daylight.

Attention uses selective focus, importance weighting, context dependence, relationships, and probabilistic models.

**Sources:**
- arxiv.org
- AK-Twitter (@akallz)
- vit-pytorch
- paperswithcode.com

We will transform the paper into a usable/deeper-get set.

**Paper overview:**
1. Get set up
2. Introduce machine learning paper replication
3. Replicate ViT for food-vision-mini
4. Train a custom ViT
5. Feature extraction with a pretrained ViT

---